In [27]:
import numpy as np

In [19]:
True and not (1 in [2,1])

False

In [9]:
def tuple_combinations(N,L):
    def helper(current_tuple, index):
        
        if index == N:
            result.append(current_tuple)
            return
        if index ==0:
            for i in range(L):
                helper(current_tuple + (i,), index+1)
        else:
            for i in range(current_tuple[-1]+1, L):
                helper(current_tuple + (i,), index+1) 
    result = []
    helper(tuple(), 0)
    return result

#all configurations that have a gap (i.e. no particles) in [j+1, k) and a particle at j
def gap_subC(j, k, Config):
    sC = []
    gap = True
    for con in Config:
        gap = True
        if j in con:
            for x in range(j+1,k):
                gap = gap and not x in con
        else:
            gap= False
        if gap:
            sC.append(con)
    return sC

#all indexes configurations that have a gap (i.e. no particles) in [j+1, k) and a particle at j
def gap_subCv2(j, k, Config):
    sC = []
    N= len(Config)
    gap = True
    for n in range(N):
        gap = True
        if j in Config[n]:
            for x in range(j+1, k):
                gap = gap and not x in Config[n]
        else:
            gap = False
        if gap:
            sC.append(n)
    return sC

def gap_subCv3(j, k, Config):
    sC = [[] for x in range(j, k)]
    N = len(Config)
    for n in range(N):
        last = Config[n][-1]
        if last >= j and last < k:
            sC[last-j].append(n)
    return sC


In [8]:
[[]  for x in range(1, 3)]

[[], []]

In [11]:
all_C = tuple_combinations(2,5)
all_C

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4)]

In [45]:
gap_subC(2,5,all_C)

[(0, 2), (1, 2)]

In [46]:
gap_subCv2(2,5,all_C)

[1, 4]

In [12]:
gap_subCv3(0,5, all_C)

[[], [0], [1, 4], [2, 5, 7], [3, 6, 8, 9]]

In [32]:
np.arange(0,4)

array([0, 1, 2, 3])

In [42]:
import json

l = [[1,2,3],[4,5,6]]

with open('data.json', 'w+') as file:
    json.dump(l, file)


OSError: [Errno 30] Read-only file system: 'data.json'

In [15]:
print("This computation took {} seconds.".format([1,2]))

This computation took [1, 2] seconds.


In [7]:
import numpy as np
from itertools import permutations
import sympy
from sympy import symbols
from sympy.matrices import Matrix
from sympy import lambdify

In [4]:
# Entries of the Guadin-like matrix
def M(sym, i, j, d):
    return d*( 1 + sym[i]*sym[i] - d*sym[i] )/((1 + sym[i]*sym[j] - d*sym[j])*(1 + sym[i]*sym[j] - d*sym[i]))

# Gaudin-like matrix
def Cmat(sym, d, L):
    n = len(sym)
    diag =0
    mat = [[ - M(sym, i, j, d) for i in range(n)] for j in range(n) ]
    for i in range(n):
        diag = sum(mat[i]) +M(sym, i, i, d)
        mat[i][i] = -diag + (L/sym[i])
    return(mat)

# Coefficients for to write the configuration basis as a linear combination of Bethe vectors
def Coeff_v2(y, z, d, L):
    n = len(z)
    terms = [ z[i]**(y[i]+1) for i in range(n)] 
    coeff = np.prod(terms)
    mat = sympy.Matrix(Cmat(z, d, L))
    coeff = coeff*mat.det()
    coeff = 1/coeff
    return coeff

# All the inversion of a permuation s
def inversions(s):
    n = len(s)
    inv = []
    for i in range(n):
        for j in range(i+1, n):
            if s[i] > s[j]:
                inv.append((s[i],s[j]))
    return inv

# Amplitude coefficients for the Bethe vectors
def A_v2(z, s, d):
    n = len(z)
    terms = [ np.where(s[i] > s[j],-(1 + z[s[i]]*z[s[j]] - d*z[s[i]])/(1 + z[s[i]]*z[s[j]] - d*z[s[j]]) ,1 ) for i in range(n) for j in range(i+1, n)]
    return np.prod(terms).item()

def All_amps(z, d):
    n = len(z)
    per = list(permutations(range(n)))
    amps = [(s, A_v2(z, s, d) ) for s in per]
    return dict(amps)

def eigenfun_v2(x, z, d, all_amps):
    n = len(z)
    per = list(permutations(range(n)))
    final_term=0
    term = 0
    for s in per:
        term = all_amps[s]
        for i in range(n):
            term = term*(z[s[i]]**(x[i]))
        final_term = final_term + term
    return final_term


def sum_terms_v2(y, aS, aC, d, L):
    n = len(y)
    sx = symbols(['x{}'.format(k) for k in range(n)])
    sz = symbols(['z{}'.format(k) for k in range(n)])
    amps = All_amps(sx, d)
    f = Coeff_v2(y, sz, d, L)*eigenfun_v2(sx, sz, d,amps)
    return f
    #return [[complex(f.subs(x).subs(z)) for x in aC] for z in aS]

def merge(list1, list2):

    merged_list = tuple(zip(list1, list2))
    return merged_list

def tuple_combinations_subs(sx,N,L):
    def helper(current_tuple, index):   
        if index == N:
            result.append(merge(sx, current_tuple))
            return
        if index ==0:
            for i in range(L):
                helper(current_tuple + (i,), index+1)
        else:
            for i in range(current_tuple[-1]+1, L):
                helper(current_tuple + (i,), index+1) 
    result = []
    helper(tuple(), 0)
    return result

def tuple_combinations(N,L):
    def helper(current_tuple, index):
        
        if index == N:
            result.append(current_tuple)
            return
        if index ==0:
            for i in range(L):
                helper(current_tuple + (i,), index+1)
        else:
            for i in range(current_tuple[-1]+1, L):
                helper(current_tuple + (i,), index+1) 
    result = []
    helper(tuple(), 0)
    return result

def BE(z,i,d):
    n = len(z)
    term=1
    for j in range(n):
        term= term*(-1)*((1 + z[i]*z[j] - d*z[i])/(1 + z[i]*z[j] - d*z[j]))
    return -term

def seq_update(z,d,L):
    n = len(z)
    w = []
    for i in range(n):
        pol = [1]
        pol.extend([0]*(L-1))
        pol.append(- BE(z, i, d))
        roots = np.roots(pol)
        roots = [x-z[i] for x in roots]
        w.append(min(roots, key=np.abs) + z[i])
    return w

def BE_sol_initial(I,L):
    n = len(I)
    pol = [1]
    pol.extend([0]*(L-1))
    pol.append((-1)**n)
    roots = np.roots(pol)
    sol =[]
    for i in I:
        sol.append(roots[i])
    return sol

def BE_all_sol(N,L,d,updates):
    all_tuples = tuple_combinations(N,L)
    sol = []
    for I in all_tuples:
        initial = BE_sol_initial(I,L)
        for k in range(updates):
            initial = seq_update(initial, d, L)
        sol.append(initial)
    return sol

def BE_all_sol_sub(N,L,d,updates, sz):
    ord_sol = BE_all_sol(N,L, d, updates)
    all_sol = []
    for z in ord_sol:
        all_sol.extend(list(permutations(z)))
    return [merge(sz, sol) for sol in all_sol]


In [5]:
n=2
L=21
d=0.1
IC_one = [2,3]
sz =symbols(['z{}'.format(k) for k in range(n)])
sx =symbols(['x{}'.format(k) for k in range(n)])
acc = 2

all_sol = BE_all_sol_sub(n,L, d, acc,sz)
all_C = tuple_combinations_subs(sx, n, L)

fun = sum_terms_v2(IC_one, all_sol, all_C, d, L)

In [8]:
lam_fun = lambdify(sz + sx, fun)

In [9]:
lam_fun(1,2,3,4)

0.002273960124222418

In [11]:
all_C2 = tuple_combinations( n, L)
sols = BE_all_sol(n,L, d, acc)

In [19]:
lam_fun(*tuple(sols[0]) + all_C2[0])

(0.0006060727767726067-0.0003214096521389549j)

In [83]:
BE_all_sol_sub(2,5, 0.1, 2,s)[0]

((z0, (-0.9999005679316239-0.014101569062283548j)),
 (z1, (-0.29557487902936197+0.9553195752661928j)))

In [76]:
tuple_combinations_subs(s , 2,5)[0]

((z0, 0), (z1, 1))

In [35]:
Coeff_v2([0,1],s, 0.1, 3 )

(0.24751862577659*z0**15*z1**15 - 0.173263038043613*z0**15*z1**14 + 0.0519789114130839*z0**15*z1**13 - 0.00866315190218064*z0**15*z1**12 + 0.000866315190218064*z0**15*z1**11 - 5.19789114130838e-5*z0**15*z1**10 + 1.73263038043613e-6*z0**15*z1**9 - 2.4751862577659e-8*z0**15*z1**8 - 0.173263038043613*z0**14*z1**15 + 3.58654488750278*z0**14*z1**14 - 2.28880473255613*z0**14*z1**13 + 0.629811143288533*z0**14*z1**12 - 0.0959010915571397*z0**14*z1**11 + 0.0086995371401698*z0**14*z1**10 - 0.00046902304398406*z0**14*z1**9 + 1.38783693472934e-5*z0**14*z1**8 - 1.73263038043613e-7*z0**14*z1**7 + 0.0519789114130839*z0**13*z1**15 - 2.28880473255613*z0**13*z1**14 + 23.9905200366328*z0**13*z1**13 - 13.916573847182*z0**13*z1**12 + 3.49143214276874*z0**13*z1**11 - 0.481942055889706*z0**13*z1**10 + 0.0392756293161061*z0**13*z1**9 - 0.0018797358976263*z0**13*z1**8 + 4.86176084750378e-5*z0**13*z1**7 - 5.19789114130838e-7*z0**13*z1**6 - 0.00866315190218064*z0**12*z1**15 + 0.629811143288533*z0**12*z1**14 - 13

In [42]:
A_v2(s, (0,1), 0.1)

1

In [44]:
amps =All_amps(s, 0.1)

In [49]:
f = amps[(1,0)]

In [72]:
f.subs(((s[0], 2), (s[1],5)))

-0.972222222222222

In [67]:
float(f.subs([(s[0], 2)]).subs([(s[1],5)]))

-0.9722222222222222

In [21]:
partition = lambda L, parts: [L[n:] for n in range(len(L)//parts)]

print(partition(list(range(10, 221, 10)), 5))

[[10, 60, 110, 160, 210], [20, 70, 120, 170, 220], [30, 80, 130, 180], [40, 90, 140, 190]]


In [46]:
parts =2
i=1
j=2
L = [[1,2,3], [4,5,6], [7,8,9]]
print(len(L))
L = np.array(L)
L = L[(i-1)*(len(L)//parts): i*(len(L)//parts), (j-1)*(len(L)//parts): j*(len(L)//parts)]
L.tolist()


3


[[2]]

In [47]:
9//10

0

In [51]:
np.array([[1,2],[3,4]]).shape

(2, 2)

In [52]:
10%3

1